# APIs

APIs são mecanismos que permitem que dois componentes de software se comuniquem usando um conjunto de definições e protocolos. Por exemplo, o sistema de software do instituto meteorológico contém dados meteorológicos diários. O aplicativo meteorológico em seu telefone “fala” com este sistema por meio de APIs e mostra atualizações meteorológicas diárias no telefone.

API significa Application Programming Interface (Interface de Programação de Aplicação). No contexto de APIs, a palavra Aplicação refere-se a qualquer software com uma função distinta. A interface pode ser pensada como um contrato de serviço entre duas aplicações. Esse contrato define como as duas se comunicam usando solicitações e respostas. A documentação de suas respectivas APIs contém informações sobre como os desenvolvedores devem estruturar essas solicitações e respostas.

![image.png](images/api.jpeg)

`client`: O cliente

`api`: O garçom

`o serviço (banco de dados, web application, twitter, spotify)`: O pizzaiolo

# Web APIs

## Some Examples

### API endpoints

**NASA API**: http://api.open-notify.org/
- http://api.open-notify.org/iss-now.json
    - Documentation: http://open-notify.org/Open-Notify-API/ISS-Location-Now/

**Meme API**

- http://apimeme.com/?ref=apilist.fun
- http://apimeme.com/meme?meme=Doge&top=Analista&bottom=de_Dados
    

**Public APIs**

- https://github.com/public-apis/public-apis

## Acessando APIs via Requests

`Requests` é uma biblioteca Python que nos permite interagir com servidores [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). Essa biblioteca pode ser utilizada para baixar páginas, arquivos e **interagir com APIs**.



[Documentação - requests](https://requests.readthedocs.io)

In [ ]:
! pip3 install requests

In [1]:
import requests, time

### Utilizando `requests`

Vamos começar acessando uma API simples que nos permite extrair, a cada 5s, a posição da [ISS](https://en.wikipedia.org/wiki/International_Space_Station).

O primeiro passo é descobrir o `endpoint` desta API. O `endpoint` é o URL através do qual acessamos uma API!

In [2]:
endpoint = 'http://api.open-notify.org/iss-now.json'

Com nosso URL em mãos, podemos utilizar a biblioteca `requests` para extrair a informação da API:

In [5]:
response = requests.get(endpoint)

O que é o objeto que guardamos na variável `response`?

In [8]:
response

<Response [200]>

O objeto é do tipo `response` e ele contém o código HTTP associado ao nosso request:

![image.png](images/http_status_codes.jpeg)

Com este código podemos saber se a nossa comunicação com o servidor de certo ou não (e porque não deu certo).

https://www.tutorialspoint.com/http/http_status_codes.htm

Para extrair a informação que a API nos retornou (no caso de um sucesso), podemos utilizar o método `.json()`. 

In [9]:
response.json()

{'iss_position': {'longitude': '94.7610', 'latitude': '7.1936'},
 'message': 'success',
 'timestamp': 1678834217}

Como nosso *request* teve sucesso (status 200), o método `.json()` nos retorna o resultado como um dicionário ou uma lista:

In [10]:
resultado = response.json()
type(resultado)

dict

In [17]:
resultado.keys()

dict_keys(['iss_position', 'message', 'timestamp'])

#### Erros de HTTP

Um problema comum ao construirmos scripts que acessam API é a tratativa de erros HTTP. Caso a API não retorne o que esperamos as etapas subsequentes de nosso programa podem falhar. As tratativas de erro HTTP são fundamentais em qualquer programa que tenha extraia informações (seja de uma API, seja via web-scrapping).

Vamos simular um erro tentando acessar um endpoint inexistente:

In [18]:
response = requests.get('http://api.open-notify.org/coisas_aleatória.json')
response

<Response [404]>

Através do objeto `Response` já vemos que o status da execução não é de sucesso: 404 indica um **Client Error**. Podemos utilizar o atributo `.status_code` deste objeto para extrair o código de status diretamente:

In [21]:
response.status_code

404

In [23]:
response.status_code == 404

True

Poderíamos tratar os diferentes códigos de status através de condicionais. No entanto, como são muitos códigos diferentes e esta é uma tarefa muito comum, o objeto `Response` contém um método que realiza essa tratativa: `.raise_for_status()`

In [24]:
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: http://api.open-notify.org/coisas_aleat%C3%B3ria.json

O método `.raise_for_status()` *levanta* um erro toda vez que um código que não seja sucesso é retornado. Sendo assim, podemos utilizar um `try:` e `except:` para tratar este `HTTPError`!

In [25]:
from requests import HTTPError

try:
    response = requests.get('http://api.open-notify.org/coisas_aleatória.json')
    response.raise_for_status()
    resultado = response.json()
except HTTPError as e:
    print(f'Falha na comunicação com o servidor:\n {e}')
    resultado = None

print(resultado)

Falha na comunicação com o servidor:
 404 Client Error: Not Found for url: http://api.open-notify.org/coisas_aleat%C3%B3ria.json
None


Vamos comparar o resultado do bloco acima utilizando o *endpoint* real da API:

In [29]:
try:
    response = requests.get('http://api.open-notify.org/iss-now.json')
    response.raise_for_status()
    resultado = response.json()
except HTTPError as e:
    print(f'Falha na comunicação com o servidor:\n {e}')
    resultado = None

print(resultado)

{'iss_position': {'longitude': '114.8058', 'latitude': '-19.8265'}, 'message': 'success', 'timestamp': 1678834756}


## Utilizando APIs I

Até agora vimos como podemos acessar, extrair informações e tratar erros de uma API através de métodos e funções fundamentais. Vamos utilizar este conhecimento para construir um `DataFrame` com as posições da ISS.

In [30]:
from datetime import datetime
import pandas as pd

response = requests.get('http://api.open-notify.org/iss-now.json')
results = response.json()

A primeira etapa de qualquer construção envolvendo uma ou mais APIs é a exploração do objeto retornado.

In [31]:
type(results)

dict

A API da ISS retorna um dicionário: vamos analisar como este dicionário está estruturado:

In [32]:
results.keys()

dict_keys(['iss_position', 'message', 'timestamp'])

Vamos investigar como as informações estão estruturadas dentro destas chaves:

In [37]:
results

{'iss_position': {'longitude': '126.1510', 'latitude': '-31.4683'},
 'message': 'success',
 'timestamp': 1678835006}

In [34]:
for chave in results.keys():
    print(f"{chave}: {type(results[chave])}")

iss_position: <class 'dict'>
message: <class 'str'>
timestamp: <class 'int'>


In [36]:
results[chave]

1678835006

As chaves `message` e `timestamp` são tipos simples (`str` e `int` respectivamente). Já a chave `iss_position` é outro dicionário:

In [40]:
results['iss_position']

{'longitude': '126.1510', 'latitude': '-31.4683'}

In [39]:
results['iss_position']['longitude']

'126.1510'

O resultado desta API é muito simples - temos 3 chaves, sendo que 2 são de interesse para nós:

1. `iss_position` contém duas chaves, `latitude` e `longitude`, contendo a posição da ISS;
1. `timestamp` contém o **timestamp** associado a esta posição.

Primeiro, vamos utilizar a função `fromtimestamp()` do módulo `datetime` para converter nosso `timestamp` em um objeto `DateTime`:

In [41]:
results['timestamp']

1678835006

In [42]:
data_hora = datetime.fromtimestamp(results['timestamp'])
print(data_hora)

2023-03-14 20:03:26


Agora que vimos como transformar este resultado, podemos construir um loop para criar uma tabela com a posição da ISS a cada 10s, por 1 minuto:

In [43]:
lat = []
long = []
data_hora = []

for i in range(6):
    response = requests.get('http://api.open-notify.org/iss-now.json')
    results = response.json()

    lat.append(float(results['iss_position']['latitude']))
    long.append(float(results['iss_position']['longitude']))
    data_hora.append(datetime.fromtimestamp(results['timestamp']))

    time.sleep(10)

In [46]:
data_hora

[datetime.datetime(2023, 3, 14, 20, 8, 54),
 datetime.datetime(2023, 3, 14, 20, 9, 5),
 datetime.datetime(2023, 3, 14, 20, 9, 15),
 datetime.datetime(2023, 3, 14, 20, 9, 26),
 datetime.datetime(2023, 3, 14, 20, 9, 36),
 datetime.datetime(2023, 3, 14, 20, 9, 46)]

Agora vamos construir nosso `DataFrame` a partir das listas que criamos em nosso loop:

In [47]:
tb_iss = pd.DataFrame({'lat': lat, 'long':long, 'time':data_hora})

In [48]:
tb_iss

,lat,long,time
0,-44.2510,146.2612,2023-03-14 20:08:54
1,-44.5859,147.0412,2023-03-14 20:09:05
2,-44.9147,147.8306,2023-03-14 20:09:15
3,-45.2375,148.6295,2023-03-14 20:09:26
4,-45.5539,149.4380,2023-03-14 20:09:36
5,-45.8494,150.2169,2023-03-14 20:09:46


In [49]:
tb_iss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   lat     6 non-null      float64       
 1   long    6 non-null      float64       
 2   time    6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 272.0 bytes


## Utilizando APIs II

A API da ISS é extremamente rudimentar: ela não possui nem autenticação nem parâmetros, simplesmente retorna a posição atual da ISS. Muitas APIs são mais complexas do que isso, necessitando de autenticação e possibilitando a utilização de parâmetros:

1. **Autenticação**: muitas APIs precisam que o seu programa se autentique, ou seja, se identifique. Muitas vezes isso é feito através do site que disponibiliza a API e requer a criação de um usuário;
1. **Parâmetros**: parâmetros são informações que passamos para a API utilizadas para determinar que informação será retornada.

No próximo exemplo utilizaremos a [WeatherAPI](https://www.weatherapi.com). Esta API retorna informações climáticas para um dado local. Ela é autenticada (através de um *token*) e necessita de uma parametrização (a latitude e longitude do local para o qual queremos extrair as informações).

Primeiro vamos criar especificar todos os parâmetros necessários para a consulta à esta API:

In [51]:
token = 'bfef1b47e3b8462199b191725221801'
endpoint = 'http://api.weatherapi.com/v1/current.json'

header = {'key' : token}
query = {'q' : '-21.996160,-47.426420'}

Utilizamos dois dicionários para especificar tanto a autenticação quanto a parametrização de nosso request:

1. `header`: é um dicionário que contém apenas uma chave, `key`, na qual guardamos nosso token identificador;
1. `query`: é um dicionário que contém apenas uma chave, `q`, na qual guardamos um string de latitude e longitude.

O nome das chaves foi determinado lendo a documentação da API: https://www.weatherapi.com/docs/.

Agora vamos utilizar nossos dicionários e a função `request()` para extrair as informações metereológicas da latitude **-21,996160** e longitude **-47,426420**

In [52]:
response = requests.request('GET', endpoint,
                            headers=header, 
                            params=query)
results = response.json()

In [53]:
response

<Response [200]>

Utilizamos o argumento `headers =` para passar ao `request()` nossos parâmetros de autenticação, e o argumento `params =` para passar os parâmetros de busca.

In [54]:
results

{'location': {'name': 'Pirassununga',
  'region': 'Sao Paulo',
  'country': 'Brazil',
  'lat': -22.0,
  'lon': -47.43,
  'tz_id': 'America/Sao_Paulo',
  'localtime_epoch': 1678836326,
  'localtime': '2023-03-14 20:25'},
 'current': {'last_updated_epoch': 1678835700,
  'last_updated': '2023-03-14 20:15',
  'temp_c': 22.0,
  'temp_f': 71.6,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 4.3,
  'wind_kph': 6.8,
  'wind_degree': 160,
  'wind_dir': 'SSE',
  'pressure_mb': 1017.0,
  'pressure_in': 30.03,
  'precip_mm': 1.0,
  'precip_in': 0.04,
  'humidity': 94,
  'cloud': 75,
  'feelslike_c': 24.5,
  'feelslike_f': 76.1,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 6.5,
  'gust_kph': 10.4}}

Esta API é muito mais complexa que a API da ISS: não só pela autenticação e parametrização, mas também pela complexidade dos resultados retornados.

Primeiro, vamos construir uma aplicação simples cujo objetivo é guardar apenas as informações temperatura, humidade, pressão do ar e velocidade do vento. Para isso precisamos investigar o resultado retornado e determinar quais chaves iremos guardar:

In [61]:
# EXERCICIO
# Extrair as informações de temperatura, humidade, pressão do ar e velocidade do vento 
# para a lista de latitudes e longitudes abaixo.
lista_latlong = [
    '-23.57,-46.69', 
    '-1.34,-48.60', 
    '-7.99,-34.87', 
    '6.51,3.39',
    '33.69,-84.44', 
    '69.00, 33.13'
]

In [62]:
results.keys()

dict_keys(['location', 'current'])

In [63]:
results['location'].keys()

dict_keys(['name', 'region', 'country', 'lat', 'lon', 'tz_id', 'localtime_epoch', 'localtime'])

In [64]:
temp = []
hum = []
press = []
vel_vento = []
nome_lugar = []
token = 'bfef1b47e3b8462199b191725221801'
endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}

for latlong in lista_latlong:
    query = {'q' : latlong}
    response = requests.request('GET', endpoint, headers=header, params=query)
    results = response.json()
    temp.append(results['current']['temp_c'])
    hum.append(results['current']['humidity'])
    press.append(results['current']['pressure_mb'])
    vel_vento.append(results['current']['wind_kph'])
    nome_lugar.append(results['location']['name'])

In [65]:
nome_lugar

['Taboao Da Serra', 'Belem', 'Olinda', 'Lagos', 'East Point', 'Murmansk']

In [66]:
tb_clima = pd.DataFrame({'nome': nome_lugar, 'vel_vento' : vel_vento, 'temp' : temp})

In [67]:
tb_clima

,nome,vel_vento,temp
0,Taboao Da Serra,11.2,21.0
1,Belem,6.8,26.0
2,Olinda,13.0,28.0
3,Lagos,6.8,28.0
4,East Point,22.0,9.4
5,Murmansk,3.6,-8.0


### Utilizando `json_normalize()`

No exercicio acima, guardamos apenas um sub-conjunto das informações extraídas. No entanto, muitas vezes é mais prático guardar todas as informações para depois filtra-la.

Para simplificar este processo podemos utilizar a função `json_normalize()` da biblioteca Pandas.

In [68]:
# Verificar variável response.json()
token = 'bfef1b47e3b8462199b191725221801'

endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}
query = {'q' : '-21.99,-47.42'}


response = requests.request('GET', endpoint, headers=header, params=query)
results = response.json()
print(results)

{'location': {'name': 'Pirassununga', 'region': 'Sao Paulo', 'country': 'Brazil', 'lat': -21.99, 'lon': -47.42, 'tz_id': 'America/Sao_Paulo', 'localtime_epoch': 1678837105, 'localtime': '2023-03-14 20:38'}, 'current': {'last_updated_epoch': 1678836600, 'last_updated': '2023-03-14 20:30', 'temp_c': 22.0, 'temp_f': 71.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 160, 'wind_dir': 'SSE', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 1.0, 'precip_in': 0.04, 'humidity': 94, 'cloud': 75, 'feelslike_c': 24.5, 'feelslike_f': 76.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 6.5, 'gust_kph': 10.4}}


In [70]:
results.keys()

dict_keys(['location', 'current'])

In [72]:
results['current']['feelslike_c']

24.5

In [69]:
pd.json_normalize(results)

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Pirassununga,Sao Paulo,Brazil,-21.99,-47.42,America/Sao_Paulo,1678837105,2023-03-14 20:38,1678836600,2023-03-14 20:30,...,0.04,94,75,24.5,76.1,10.0,6.0,1.0,6.5,10.4


Primeiro vamos construir um loop para percorrer uma lista de latitudes e longitudes e armazenar os dicionários de resultados em uma lista:

In [73]:
lista_latlong = [
    '-23.57,-46.69', 
    '-1.34,-48.60', 
    '-7.99,-34.87', 
    '6.51,3.39',
    '33.69,-84.44', 
    '69.00, 33.13'
]

In [74]:
token = 'bfef1b47e3b8462199b191725221801'
endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}
lista_resultados = []
for latlong in lista_latlong:
    query = {'q' : latlong}
    response = requests.request('GET', endpoint, headers=header, params=query)
    results = response.json()
    lista_resultados.append(results)

In [76]:
lista_resultados[1]

{'location': {'name': 'Belem',
  'region': 'Para',
  'country': 'Brazil',
  'lat': -1.34,
  'lon': -48.6,
  'tz_id': 'America/Belem',
  'localtime_epoch': 1678837229,
  'localtime': '2023-03-14 20:40'},
 'current': {'last_updated_epoch': 1678836600,
  'last_updated': '2023-03-14 20:30',
  'temp_c': 26.0,
  'temp_f': 78.8,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 4.3,
  'wind_kph': 6.8,
  'wind_degree': 310,
  'wind_dir': 'NW',
  'pressure_mb': 1012.0,
  'pressure_in': 29.88,
  'precip_mm': 16.4,
  'precip_in': 0.65,
  'humidity': 89,
  'cloud': 75,
  'feelslike_c': 30.4,
  'feelslike_f': 86.7,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 0.7,
  'gust_kph': 1.1}}

A variável `lista_resultados` é uma `list` de `dicts`: cada elemento desta lista contém as informações de uma localização específica em um dicionário.

Agora, vamos utilizar a função `json_normalize()` da biblioteca `Pandas` para transformar esta lista em um `DataFrame`, onde cada linha será correspondente à cada elemento da lista:

In [77]:
tb_weather = pd.json_normalize(lista_resultados)
tb_weather


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Taboao Da Serra,Sao Paulo,Brazil,-23.57,-46.69,America/Sao_Paulo,1678837229,2023-03-14 20:40,1678836600,2023-03-14 20:30,...,0.02,94,100,21.0,69.8,10.0,6.0,1.0,7.4,11.9
1,Belem,Para,Brazil,-1.34,-48.60,America/Belem,1678837229,2023-03-14 20:40,1678836600,2023-03-14 20:30,...,0.65,89,75,30.4,86.7,10.0,6.0,1.0,0.7,1.1
2,Olinda,Pernambuco,Brazil,-7.99,-34.87,America/Recife,1678837230,2023-03-14 20:40,1678836600,2023-03-14 20:30,...,0.00,70,50,31.8,89.3,10.0,6.0,1.0,16.3,26.3
3,Lagos,Lagos,Nigeria,6.51,3.39,Africa/Lagos,1678837231,2023-03-15 0:40,1678836600,2023-03-15 00:30,...,0.00,84,25,31.9,89.3,10.0,6.0,1.0,18.3,29.5
4,East Point,Georgia,United States of America,33.69,-84.44,America/New_York,1678837231,2023-03-14 19:40,1678836600,2023-03-14 19:30,...,0.00,35,0,5.4,41.7,16.0,9.0,3.0,13.4,21.6
5,Murmansk,Murmansk,Russia,69.00,33.13,Europe/Moscow,1678837231,2023-03-15 2:40,1678836600,2023-03-15 02:30,...,0.08,85,37,-12.0,10.5,10.0,6.0,1.0,7.8,12.6


A função transformou os dicionários hierarquicos em colunas individuais: para evitar a colisão de nomes de colunas, utilizou-se a notação de `.`: por exemplo, o sub-dicionário `location`, que contém campos relativos à localização, foi armazenado nas colunas `location.name`, `location.region`, etc...

In [78]:
tb_weather.columns

Index(['location.name', 'location.region', 'location.country', 'location.lat',
       'location.lon', 'location.tz_id', 'location.localtime_epoch',
       'location.localtime', 'current.last_updated_epoch',
       'current.last_updated', 'current.temp_c', 'current.temp_f',
       'current.is_day', 'current.condition.text', 'current.condition.icon',
       'current.condition.code', 'current.wind_mph', 'current.wind_kph',
       'current.wind_degree', 'current.wind_dir', 'current.pressure_mb',
       'current.pressure_in', 'current.precip_mm', 'current.precip_in',
       'current.humidity', 'current.cloud', 'current.feelslike_c',
       'current.feelslike_f', 'current.vis_km', 'current.vis_miles',
       'current.uv', 'current.gust_mph', 'current.gust_kph'],
      dtype='object')

In [79]:
tb_weather.describe()

,location.lat,location.lon,location.localtime_epoch,current.last_updated_epoch,current.temp_c,current.temp_f,current.is_day,current.condition.code,current.wind_mph,current.wind_kph,...,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
count,6.000000,6.000000,6.000000e+00,6.000000e+00,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000,6.000000,6.000000
mean,12.716667,-29.680000,1.678837e+09,1.678837e+09,17.216667,62.983333,0.166667,1003.500000,6.383333,10.266667,...,0.125000,76.166667,47.833333,18.083333,64.550000,11.00000,6.500000,1.333333,10.650000,17.166667
std,33.434399,41.741158,9.831921e-01,0.000000e+00,14.418102,25.957536,0.408248,2.949576,3.655361,5.924413,...,0.259056,21.701766,35.751457,17.917189,32.208182,2.44949,1.224745,0.816497,6.568029,10.601069
min,-23.570000,-84.440000,1.678837e+09,1.678837e+09,-8.000000,17.600000,0.000000,1000.000000,2.200000,3.600000,...,0.000000,35.000000,0.000000,-12.000000,10.500000,10.00000,6.000000,1.000000,0.700000,1.100000
25%,-6.327500,-48.122500,1.678837e+09,1.678837e+09,11.475000,52.625000,0.000000,1003.000000,4.300000,6.800000,...,0.000000,73.500000,28.000000,9.300000,48.725000,10.00000,6.000000,1.000000,7.500000,12.075000
50%,2.585000,-40.780000,1.678837e+09,1.678837e+09,23.500000,74.300000,0.000000,1003.000000,5.600000,9.000000,...,0.010000,84.500000,43.500000,25.700000,78.250000,10.00000,6.000000,1.000000,10.600000,17.100000
75%,26.895000,-6.175000,1.678837e+09,1.678837e+09,27.500000,81.500000,0.000000,1003.000000,7.800000,12.550000,...,0.065000,88.000000,68.750000,31.450000,88.650000,10.00000,6.000000,1.000000,15.575000,25.125000
max,69.000000,33.130000,1.678837e+09,1.678837e+09,28.000000,82.400000,1.000000,1009.000000,12.500000,20.200000,...,0.650000,94.000000,100.000000,31.900000,89.300000,16.00000,9.000000,3.000000,18.300000,29.500000


# Encapsulando APIs 

Muitas vezes as funções de uma API são encapsuladas em um **wrapper**: uma classe que contém todo o código necessário para autenticar e parametrizar queries em uma dada API. Vamos construir um **wrapper** para a nossa tarefa acima.

In [81]:
class WeatherAPI:
    '''
    Wrapper para a API https://www.weatherapi.com/docs/
    '''
    def __init__(self, token):
        '''
        Inicializa o wrapper.
        token Str: Token de identificação do usuário da API
        '''
        self.header = {'key' : token}

    def query_latlong(self, lat, long):
        '''
        Busca dados meterológicos atuais para uma localização específica
        lat Float: latitude da localização
        long Float: longitude da localização
        '''
        query = {'q' : f"{lat:.4f},{long:.4f}"}
        try:
            response = requests.request('GET', endpoint, 
                                        headers=self.header, 
                                        params=query)
            response.raise_for_status()
            return response.json()
        except HTTPError as e:
            print(e)
            return None
    

In [82]:
minha_api = WeatherAPI('bfef1b47e3b8462199b191725221801')

In [84]:
minha_api.header

{'key': 'bfef1b47e3b8462199b191725221801'}

In [86]:
minha_api.query_latlong(-15, 49)

{'location': {'name': 'Bezono',
  'region': 'Mahajanga',
  'country': 'Madagascar',
  'lat': -15.0,
  'lon': 49.0,
  'tz_id': 'Indian/Antananarivo',
  'localtime_epoch': 1678837736,
  'localtime': '2023-03-15 2:48'},
 'current': {'last_updated_epoch': 1678837500,
  'last_updated': '2023-03-15 02:45',
  'temp_c': 18.7,
  'temp_f': 65.7,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 3.6,
  'wind_kph': 5.8,
  'wind_degree': 102,
  'wind_dir': 'ESE',
  'pressure_mb': 1012.0,
  'pressure_in': 29.87,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 88,
  'cloud': 30,
  'feelslike_c': 18.7,
  'feelslike_f': 65.7,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 5.6,
  'gust_kph': 9.0}}